# Digital Twin Data Acquisition

In this notebook, we will use Big Query and The Graph to obtain the signals required for the Rai Digital Twin. We will get state snapshots and calculate the absolute changes between time periods. 


## Mapping of specification states to data
The quantity state variables of the system are:
* ETH in collateral = collateral
* ETH in Uniswap = reserve1
* RAI in Uniswap = reserve0 
* RAI drawn from SAFEs = erc20CoinTotalSupply

The metric state variables of the system are:

* Market Price of RAI in ETH = marketPriceEth
* Market Price of RAI in USD = marketPriceUsd
* Market Price of ETH in USD = ETH Price (OSM)

The metric control variables of the system are:

* Redemption Price of RAI in USD = RedemptionPrice
* Redemption Price of RAI in ETH = RedemptionPriceinEth

In [1]:
# import libraries
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import json
import requests
import os
# The Graph URL
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

%matplotlib inline

# constants
constant = 1000000000000000000

#defining creditionals
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/aclarkdata/Downloads/raidata-5e1723d42cf5.json'

# initializing agent
client = bigquery.Client()


In [2]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.ExternallyFundedOSM_event_UpdateResult`
ORDER By block_timestamp ASC

"""

eth_price_OSM = client.query(sql).to_dataframe()
eth_price_OSM['ETH Price (OSM)'] = eth_price_OSM['newMedian'].astype(float)/constant
# subset
eth_price_OSM = eth_price_OSM[['block_number','ETH Price (OSM)','block_timestamp']]

eth_price_OSM.head()

,block_number,ETH Price (OSM),block_timestamp
0,12371493,3349.780000,2021-05-05 02:23:19+00:00
1,12371765,3349.780000,2021-05-05 03:22:45+00:00
2,12372918,3297.580000,2021-05-05 07:34:45+00:00
3,12373045,3274.547289,2021-05-05 08:04:07+00:00
4,12373322,3342.258435,2021-05-05 09:03:55+00:00


In [3]:
eth_price_OSM.tail()

,block_number,ETH Price (OSM),block_timestamp
425,12584395,2702.280000,2021-06-07 01:05:54+00:00
426,12585445,2757.389589,2021-06-07 05:07:45+00:00
427,12586509,2773.590000,2021-06-07 09:06:20+00:00
428,12587589,2761.044990,2021-06-07 13:15:51+00:00
429,12588614,2828.150972,2021-06-07 17:06:11+00:00


In [4]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.OSM_event_UpdateResult`
WHERE block_number < 12371493
ORDER By block_timestamp ASC

"""

eth_price_OSM_old = client.query(sql).to_dataframe()
eth_price_OSM_old['ETH Price (OSM)'] = eth_price_OSM_old['newMedian'].astype(float)/constant
# subset
eth_price_OSM_old = eth_price_OSM_old[['block_number','ETH Price (OSM)','block_timestamp']]

eth_price_OSM_old.head()

,block_number,ETH Price (OSM),block_timestamp
0,11848351,1803.656430,2021-02-13 12:46:18+00:00
1,11860755,1803.656430,2021-02-15 10:16:16+00:00
2,11861008,1763.974936,2021-02-15 11:13:28+00:00
3,11861237,1763.974936,2021-02-15 12:07:36+00:00
4,11861502,1775.977923,2021-02-15 13:07:59+00:00


In [5]:
eth_price_OSM_old.tail()

,block_number,ETH Price (OSM),block_timestamp
1885,12370391,3390.623840,2021-05-04 22:09:45+00:00
1886,12370625,3347.555806,2021-05-04 23:10:04+00:00
1887,12370903,3301.810000,2021-05-05 00:10:50+00:00
1888,12371173,3264.914896,2021-05-05 01:09:41+00:00
1889,12371432,3334.690000,2021-05-05 02:10:02+00:00


In [6]:
eth_price = eth_price_OSM_old.append(eth_price_OSM)

In [7]:
eth_price

,block_number,ETH Price (OSM),block_timestamp
0,11848351,1803.656430,2021-02-13 12:46:18+00:00
1,11860755,1803.656430,2021-02-15 10:16:16+00:00
2,11861008,1763.974936,2021-02-15 11:13:28+00:00
3,11861237,1763.974936,2021-02-15 12:07:36+00:00
4,11861502,1775.977923,2021-02-15 13:07:59+00:00
...,...,...,...
425,12584395,2702.280000,2021-06-07 01:05:54+00:00
426,12585445,2757.389589,2021-06-07 05:07:45+00:00
427,12586509,2773.590000,2021-06-07 09:06:20+00:00
428,12587589,2761.044990,2021-06-07 13:15:51+00:00


In [8]:
blocknumbers = eth_price.block_number.values.tolist()


In [9]:
uniswap_url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
pair = []
for i in blocknumbers:
    query = '''
    {
      pairs(block: {number:%s}, where: {id: "0x8ae720a71622e824f576b4a8c03031066548a3b1"}){
        id,
        token0 {
          id
        },
        token1 {
          id
        }
        reserve0,
        reserve1,
        totalSupply,
        reserveETH,
        reserveUSD,
        token0Price,
        token1Price
      }
    }
    
    ''' % i
    r = requests.post(uniswap_url, json = {'query':query})
    s = json.loads(r.content)['data']['pairs']
    pair.append(s)
        
pairState = pd.DataFrame(pair)

pairState['block_number'] = blocknumbers
pairState['data'] = pairState[0]
del pairState[0]
pairState = pairState[1:]


pairState['EthInUniswap'] = pairState.data.apply(lambda x: x['reserve1'])
pairState['RaiInUniswap'] = pairState.data.apply(lambda x: x['reserve0'])

# subset 
pairState = pairState[['block_number','EthInUniswap','RaiInUniswap']]
pairState.tail()



,block_number,EthInUniswap,RaiInUniswap
2315,12584395,6564.157346459202112906,6014011.618217051683984889
2316,12585445,6527.685380199155840328,6047050.67719801013286767
2317,12586509,6565.000884794524909542,6039329.122833301597860332
2318,12587589,6495.694866208574513864,6107233.207612577182605722
2319,12588614,6541.01895026488098977,6065035.164579718018465007


In [10]:
test  = pd.DataFrame(pair)
test['block_number'] = blocknumbers
test['data'] = test[0]
del test[0]
test = test[1:]
test

,block_number,data
1,11860755,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2,11861008,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
3,11861237,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
4,11861502,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
5,11861791,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
...,...,...
2315,12584395,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2316,12585445,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2317,12586509,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2318,12587589,{'id': '0x8ae720a71622e824f576b4a8c03031066548...


In [11]:
test[test['block_number'] == 12345529]['data'].values[0]

{'id': '0x8ae720a71622e824f576b4a8c03031066548a3b1',
 'reserve0': '18908049.836669973479224348',
 'reserve1': '20412.543157392702619562',
 'reserveETH': '40825.08631478540523912400000000001',
 'reserveUSD': '115195549.3881934103316527484790104',
 'token0': {'id': '0x03ab458634910aad20ef5f1c8ee96f1d6ac54919'},
 'token0Price': '926.2956453234561924515887747803531',
 'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'},
 'token1Price': '0.001079568931419090007655332093230422',
 'totalSupply': '599054.284772156919258854'}

In [12]:
state = []
for i in blocknumbers:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          label
          reserve0
          reserve1
          token0Price
          token1Price
          totalSupply
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        safeCount,
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
        lastPeriodicUpdate
        createdAt
        createdAtBlock
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState['block_number'] = blocknumbers

systemState = systemState[1:]

systemState.head()


,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,1613338681,3,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11860755
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,1613384616,3,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861008
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,1613384616,3,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861237
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613393080,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861502
5,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613393080,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861791


In [13]:
## reserve 0= rai
## reserve 1 = eth

# The quantity state variables of the system are:
# * ETH in collateral = collateral
# * ETH in Uniswap = reserve1
# * RAI in Uniswap = reserve0 
# * RAI drawn from SAFEs = erc20CoinTotalSupply
systemState.coinUniswapPair.values[-100:-99][0]

In [14]:
systemState.coinUniswapPair.values[-1]

{'label': 'UNISWAP_POOL_COIN',
 'reserve0': '6065035.164579718018465007',
 'reserve1': '6541.01895026488098977',
 'token0Price': '927.2309422577215081656982523330028',
 'token1Price': '0.001078479971305845955331113621049587',
 'totalSupply': '189343.548493613835638332'}

In [15]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])    
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
# systemState['EthInUniswap'] = systemState.coinUniswapPair.apply(lambda x: x['reserve1'])
# systemState['RaiInUniswap'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
systemState['RaiDrawnFromSAFEs'] = systemState['erc20CoinTotalSupply']
#systemState['RAIInUniswapV2(RAI/ETH)'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']


# subset

systemState = systemState[['debtAvailableToSettle','globalDebt',
                           'globalDebtCeiling','systemSurplus','totalActiveSafeCount',
                           'block_number', 'RedemptionRateAnnualizedRate',
                           'RedemptionRateHourlyRate', 'RedemptionRateEightHourlyRate',
                        'RedemptionPrice','RaiDrawnFromSAFEs']]

In [16]:
hourly = []
for i in blocknumbers:
    query = '''
    {
      hourlyStats(where: {blockNumber_gt: %s}) { 
        marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
        marketPriceEth # Price of COIN in ETH (uni pool price)
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats'][0]
    hourly.append(s)
        
hourlyStats = pd.DataFrame(hourly)

hourlyStats['block_number'] = blocknumbers

hourlyStats.head()


,marketPriceEth,marketPriceUsd,block_number
0,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,11848351
1,0.002481355746535171105569862943787003,4.362594137056344705089877147817243,11860755
2,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861008
3,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861237
4,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861502


In [18]:
safehistories = []
for i in blocknumbers[1:]:
    query = '''
    {
      safes(block: {number:%s}) {
            collateral
            debt
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    t = pd.DataFrame(s)
    t['collateral'] = t['collateral'].astype(float)
    t['debt'] = t['debt'].astype(float)
    safehistories.append(pd.DataFrame(t.sum().to_dict(),index=[0]))

safe_history = pd.concat(safehistories)


In [19]:
safehistories = []
for i in blocknumbers[1:]:
    query = '''
    {
      safes(block: {number:%s}) {
            collateral
            debt
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    t = pd.DataFrame(s)
    t['collateral'] = t['collateral'].astype(float)
    t['debt'] = t['debt'].astype(float)
    safehistories.append(pd.DataFrame(t.sum().to_dict(),index=[0]))

safe_history = pd.concat(safehistories)



In [20]:
safe_history.head()

,collateral,debt
0,14.89236,2788.385291
0,14.89236,2788.385291
0,14.89236,2788.385291
0,16.89236,3048.358490
0,16.89236,3048.358490


In [21]:
safe_history['block_number'] = blocknumbers[1:]
safe_history.reset_index(inplace=True)
del safe_history['index']

In [22]:
states = hourlyStats.merge(systemState,how='inner',on='block_number')
states = states.merge(safe_history,how='inner',on='block_number')
states = states.merge(pairState,how='inner',on='block_number')

In [23]:
states = states.merge(eth_price,how='inner',on='block_number')

In [24]:
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RaiDrawnFromSAFEs,collateral,debt,EthInUniswap,RaiInUniswap,ETH Price (OSM),block_timestamp
0,0.002481355746535171105569862943787003,4.362594137056344705089877147817243,11860755,0,2788.522357955470745187436691184972,115792089237316195423570985008687.9,0.06853325908830422221834559248594176,3,1,1,1,3.14,2788.440432971667032725,14.89236,2788.385291,1.679713392792866803,676.933726709008306816,1803.656430,2021-02-15 10:16:16+00:00
1,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861008,0,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,1,1,1,3.14,2788.440432971667032725,14.89236,2788.385291,1.679713392792866803,676.933726709008306816,1763.974936,2021-02-15 11:13:28+00:00
2,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861237,0,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,1,1,1,3.14,2788.440432971667032725,14.89236,2788.385291,1.679713392792866803,676.933726709008306816,1763.974936,2021-02-15 12:07:36+00:00
3,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861502,0,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1,1,1,3.14,3048.440432971667032725,16.89236,3048.358490,1.418568601331794594,801.933726709008306816,1775.977923,2021-02-15 13:07:59+00:00
4,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861791,0,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1,1,1,3.14,3048.440432971667032725,16.89236,3048.358490,1.418568601331794594,801.933726709008306816,1805.792735,2021-02-15 14:07:59+00:00


In [25]:
states.tail(10)

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RaiDrawnFromSAFEs,collateral,debt,EthInUniswap,RaiInUniswap,ETH Price (OSM),block_timestamp
2309,0.00112240697297394549615101797283226,3.009869913772518142732162447740107,12579185,0,11254666.38987890039495928507191416,115792089237316195423570985008687.9,334422.2097696048411166656167537195,398,1.0280557716144304,1.0000031586143612,1.000025269194242,3.007462336651507650240484082,10829646.548524888029772788,1083.900454,250083.541093,6625.838114615076905674,5943514.593703697883733875,2683.690690,2021-06-06 06:05:41+00:00
2310,0.001119742851210339752900229042003111,2.997289250994206395982904533877571,12580301,0,11255257.42502759395701312239432585,115792089237316195423570985008687.9,334495.6948952217890414417844415978,398,0.9917512017587372,0.9999990544516076,0.9999924356378951,3.007482317021276222905288762,10830146.548524888029772788,1083.900454,250083.541093,6589.450447779259016653,5976558.639784811539357153,2707.672804,2021-06-06 10:05:55+00:00
2311,0.001108387134859624426138202562191223,2.987795712618607961022027524294928,12581396,0,11254262.85066739514570282239432585,115792089237316195423570985008687.9,334495.6948952217890414417844415978,397,0.9917512017587372,0.9999990544516076,0.9999924356378951,3.007470946112948759641228752,10829145.874404995566708985,1083.900454,250083.541093,6637.300038771896535518,5932304.599899358670044537,2725.020000,2021-06-06 14:05:50+00:00
2312,0.001108387134859624426138202562191223,2.987795712618607961022027524294928,12582289,0,11254262.85066739514570282239432585,115792089237316195423570985008687.9,334495.6948952217890414417844415978,397,0.9848502766846086,0.9999982573470972,0.999986058861809,3.007457913531166416987794407,10829145.874404995566708985,1083.900454,250083.541093,6595.238341540219075928,5970269.322165615521018742,2664.550000,2021-06-06 17:35:43+00:00
2313,0.001110815700425048432070674732878301,3.01484838493587938693177994526334,12583291,0,11269458.84471393850414366224594458,115792089237316195423570985008687.9,334641.1279151782159872447426343482,398,0.9848502766846086,0.9999982573470972,0.999986058861809,3.007442141195426393007003677,10844145.874404995566708985,1083.900454,250083.541093,6624.69863615400826378,5958843.293272671861700994,2700.841159,2021-06-06 21:05:55+00:00
2314,0.00108158676228900473873979918684649,3.017312488066069800526750719988266,12584395,0,11273343.52215627828906702101612578,115792089237316195423570985008687.9,334713.0306017743527719623165356281,397,1.004843332270226,1.000000551557352,1.000004412467334,3.007421877781552256694398164,10847800.017651307208637556,1083.900454,250083.541093,6564.157346459202112906,6014011.618217051683984889,2702.280000,2021-06-07 01:05:54+00:00
2315,0.001079482499594968644440033313078683,3.010808833251978593543698264644376,12585445,0,11273343.52215627828906702101612578,115792089237316195423570985008687.9,334713.0306017743527719623165356281,397,1.004843332270226,1.000000551557352,1.000004412467334,3.007429604409038912451083644,10847800.017651307208637556,1083.900454,250083.541093,6527.685380199155840328,6047050.67719801013286767,2757.389589,2021-06-07 05:07:45+00:00
2316,0.001085634109767479233156514044862049,3.013380911523254671540900690877811,12586509,0,11271749.17020874565881833601612578,115792089237316195423570985008687.9,334713.0306017743527719623165356281,395,0.9733785853054229,0.9999969198470134,0.9999753590417518,3.007431267158340112904837268,10846195.825558383918809618,1083.900454,250083.541093,6565.000884794524909542,6039329.122833301597860332,2773.590000,2021-06-07 09:06:20+00:00
2317,0.001076494745281016098063717208212789,2.994557766505518994467129012274117,12587589,0,11280625.25524269030657278130476457,115792089237316195423570985008687.9,334865.7694774930324295870965585239,395,0.9733785853054229,0.9999

In [26]:
os.chdir('..')
# export
states.to_csv('data/states.csv')